In [1]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
import torch
import numpy as np
import time
from utils import HDDataset
from model import BModel
from torchvision import datasets, transforms
import scipy
from encoder_1d import RandomFourierEncoder
import main2_1d as main2 
import importlib
importlib.reload(main2)
import csv
from scipy.io import savemat, loadmat
from torch.utils.data import DataLoader, Subset, TensorDataset
from aeon.datasets import load_classification
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt
from tqdm import tqdm


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
dataset_name = 'WalkingSittingStanding'
# Load dataset
X_train, y_train, metadata = load_classification(dataset_name, return_metadata=True, split='train')
X_test, y_test = load_classification(dataset_name, split='test')
if X_train.shape[0] < 200:
    if X_test.shape[0] >= 200:
        train_size = (X_train.shape[0] + X_test.shape[0]) * 3/4
        x, y = load_classification(dataset_name)
        X_train, y_train = x[:train_size, :], y[:train_size]
        X_test, y_test = x[train_size:, :], y[train_size:]

In [4]:
input_channels = 1
if X_train.ndim == 3:
    input_channels = X_train.shape[1]
seq_length = X_train.shape[-1]
if y_train.dtype == object or isinstance(y_train[0], str):
    le = LabelEncoder()
    y_train = le.fit_transform(y_train)
    y_test = le.transform(y_test)

In [5]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(X_train.shape[0], -1))
X_test_scaled = scaler.transform(X_test.reshape(X_test.shape[0], -1))


X_min = X_train_scaled.min(axis=0)
X_max = X_train_scaled.max(axis=0)

denom = (X_max - X_min)
denom[denom == 0] = 1   # avoid division by zero

X_train_norm = (X_train_scaled - X_min) / denom
X_test_norm  = (X_test_scaled  - X_min) / denom

# Optional: clip to [0,1] just in case
X_train_norm = np.clip(X_train_norm, 0, 1)
X_test_norm  = np.clip(X_test_norm, 0, 1)
X_train_tensor = torch.tensor(X_train_norm, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test_norm, dtype=torch.float32).to(device)

In [6]:
X_train_tensor.shape

torch.Size([7352, 618])

In [7]:
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

num_classes = len(np.unique(y_train))

In [8]:
torch.min(X_test_tensor), torch.max(X_test_tensor), np.unique(y_train)

(tensor(0., device='cuda:0'),
 tensor(1., device='cuda:0'),
 array([0, 1, 2, 3, 4, 5]))

In [9]:
gorder = 2
dim = 15000
gamma = 0.3
lr = 0.01
classes = 10
channels = 3
epochs = 3
train = main2.train
test = main2.test

In [10]:
X_train_tensor.shape

torch.Size([7352, 618])

In [11]:
@torch.no_grad()                       # no autograd graph needed
def flip_rows_(tensor: torch.Tensor, perc: float, *, inplace: bool = True) -> torch.Tensor:
    """
    Flip the sign of `perc` fraction of elements **in every row** of `tensor`.

    Args
    ----
    tensor : torch.Tensor               # shape (N, D)
        2-D tensor whose rows will be sign-flipped.
    perc   : float                      # 0 ≤ perc ≤ 1
        Fraction of positions per row to flip.
    inplace: bool (default True)
        If True, modify `tensor` in place and return it;
        otherwise return a flipped clone.

    Returns
    -------
    torch.Tensor
        The tensor with sign-flipped rows (same object if `inplace` is True).
    """
    if not (0.0 <= perc <= 1.0):
        raise ValueError("`perc` must be in the interval [0, 1].")

    # Short-circuit trivial cases
    if perc == 0.0 or tensor.numel() == 0:
        return tensor
    if not inplace:
        tensor = tensor.clone()

    N, D = tensor.shape
    k = int(round(D * perc))            # exact count per row
    if k == 0:
        return tensor                   # nothing to flip

    device = tensor.device
    for r in range(N):
        # choose k unique positions in this row
        idx = torch.randperm(D, device=device)[:k]
        tensor[r, idx] *= -1            # in-place sign change

    return tensor


In [12]:
n_splits = 20
split_size = len(test_data) // n_splits
flip_percs = np.arange(0.0, 0.51, 0.05)
accuracies1 = np.zeros((len(flip_percs), n_splits))
hyperdim = 15_000
input_dim = torch.prod(torch.tensor(list(train_data[0][0].size())))
print(input_dim)

for i, perc in enumerate(flip_percs):
    print(f'Flipping Percentage: {np.round(perc, 2)} ------------')
    for split_idx in range(n_splits):
        indices = list(range(len(test_data)))
        np.random.shuffle(indices)  # or random.shuffle(indices)
        start_idx = split_idx * split_size
        end_idx = start_idx + split_size
        split_indices = indices[start_idx:end_idx]
        split_subset = Subset(test_data, split_indices)
        # print(f'Round {j+1}')
        # Train
        encoder = RandomFourierEncoder(input_dim, gamma, gorder, output_dim=hyperdim)
        encoder.build_item_mem()
        train_hd, y_train = encoder.encode_data_extract_labels(train_data)
        train_dataset = HDDataset(train_hd.cpu(), y_train.cpu())
        trainloader = DataLoader(train_dataset, batch_size=10, shuffle=True, pin_memory=False, num_workers=0)
        # Test
        
        test_hd, y_test = encoder.encode_data_extract_labels(split_subset)
        flip_rows_(test_hd, perc=perc)  
        # test_hd, y_test = encoder.encode_data_extract_labels(test_data)
        test_dataset = HDDataset(test_hd, y_test)
        testloader = DataLoader(test_dataset, batch_size=10, shuffle=False, pin_memory=False, num_workers=0)
        accuracies1[i, split_idx] = train(trainloader, testloader, lr, hyperdim)
        del testloader
        del test_dataset
        del test_hd

        
    del train_hd
    del trainloader
    del train_dataset
    print(f'Average Accuracy for Flipping Percentage={perc}: {np.mean(accuracies1[i, :])}')

tensor(618)
Flipping Percentage: 0.0 ------------
Average Accuracy for Flipping Percentage=0.0: 70.40816326530611
Flipping Percentage: 0.05 ------------
Average Accuracy for Flipping Percentage=0.05: 70.7482993197279
Flipping Percentage: 0.1 ------------
Average Accuracy for Flipping Percentage=0.1: 69.35374149659866
Flipping Percentage: 0.15 ------------
Average Accuracy for Flipping Percentage=0.15000000000000002: 68.46938775510203
Flipping Percentage: 0.2 ------------
Average Accuracy for Flipping Percentage=0.2: 71.46258503401359
Flipping Percentage: 0.25 ------------
Average Accuracy for Flipping Percentage=0.25: 66.59863945578232
Flipping Percentage: 0.3 ------------
Average Accuracy for Flipping Percentage=0.30000000000000004: 66.02040816326532
Flipping Percentage: 0.35 ------------
Average Accuracy for Flipping Percentage=0.35000000000000003: 63.9795918367347
Flipping Percentage: 0.4 ------------
Average Accuracy for Flipping Percentage=0.4: 56.666666666666664
Flipping Percenta

In [13]:
np.mean(accuracies1, axis=1)

array([70.40816327, 70.74829932, 69.3537415 , 68.46938776, 71.46258503,
       66.59863946, 66.02040816, 63.97959184, 56.66666667, 38.06122449,
       13.26530612])

In [14]:
savemat(f'{dataset_name}_RFFHDC.mat', {f'{dataset_name}_RFFHDC': accuracies1})